In [ ]:
%load_ext autoreload
%autoreload 2

from repo import *
import utils as u
import numpy as np
import seaborn as sns
from sklearn.metrics import roc_curve, auc

In [2]:
# get the file names
train_files, test_files = u.train_test_all_ids('/data/home/shruti/voxceleb/vgg/leaders/')
all_lbls = list(train_files.keys())
all_lbls.sort()
all_lbls = np.array(all_lbls)

before 4105
after 3739


In [5]:

# repo with leaders + ff original + imposters + Google
def get_train_repo(params):
    
    out_repo = Repo(params['bsfldr'], params['frames'], params['steps'], np.mean, 5000)
    out_repo.add_ids({f: train_files[f] for f in ['bo','br','bs','cb','dt','ew','hc','jb','kh', 'pb',
                                                  'bo_imposter','bs_imposter','ew_imposter','dt_imposter',
                                                   'hc_imposter', 'jb_imposter']}, 0, 1)
    out_repo.add_ids({f: train_files[f] for f in train_files.keys() if 'FF_' in f}, 0, 0.5)
    out_repo.add_ids({f: train_files[f] for f in train_files.keys() if 'GG_' in f}, 0, 1)
    out_repo.add_ids({'steve_b': train_files['steve_b']}, 0, 1)
    out_repo.add_ids({'jen_l': train_files['jen_l']}, 0, 1)
    
    # get dfdc dataset id
    out_repo.add_ids({k: train_files[k] for k in train_files.keys() if 'DFDC_' in k}, 0, 1)
    out_repo.build_repo_noKDD()
    
    return out_repo


def save_results(vgg_repo, fab_repo, test_ids, r_or_f, N_f, out_file, n1, n2, parallel=False):
        
    # number of files per id
    test_dict = {}
    for ids in test_ids:
        cur_test_files = test_files[r_or_f][ids]
        if N_f>0 and len(cur_test_files)>N_f:
            test_dict[ids] = cur_test_files[:N_f].copy()
        else:
            test_dict[ids] = cur_test_files.copy()
            
    vgg_result = vgg_repo.dist_using_dict(test_dict, n1, n2, parallel=parallel, dist='cosine')
    vgg_result['RealFake'] = r_or_f
    fab_result = fab_repo.dist_using_dict(test_dict, n1, n2, parallel=parallel, dist='cosine')
    fab_result['RealFake'] = r_or_f
    full_df = vgg_result.set_index('fileName').join(fab_result.set_index('fileName'), 
                                                 how='inner', lsuffix='_vgg', rsuffix='_fab')
    full_df = full_df[['predLabel_vgg', 'actualLabel_vgg', 
                       'RealFake_vgg', 'predLabel_fab', 'dist_vgg', 'dist_fab']].copy()
    
    if out_file is not None:
        full_df.to_csv(out_file)
        
    return full_df

# No Time

### Repo of Leaders + Face Forensics Original + Imposter + Google

In [ ]:
vgg_repo = get_train_repo({'istrain':True, 'bsfldr':'/data/home/shruti/voxceleb/vgg/leaders/', 
                                'frames':1, 'steps':5, 'N_comp':512})

fab_repo = get_train_repo({'istrain':True, 'bsfldr':'/data/home/shruti/voxceleb/fabnet/leaders/', 
                                'frames':1, 'steps':5, 'N_comp':-1})


In [ ]:
save_results(vgg_repo, fab_repo, 
             ['FF_{0:03d}'.format(f) for f in range(1000)], 
             'real', -1, 
             'result_csv/FF_real_notime.csv', 0.5, 1, parallel=True)
save_results(vgg_repo, fab_repo, 
             ['FF_{0:03d}'.format(f) for f in range(1000)], 
             'fake', -1, 
             'result_csv/FF_fake_notime.csv', 0.5, 1, parallel=True)

In [ ]:
# change the location of the base folder in the repo
vgg_repo.bs_fldr = '/data/home/shruti/voxceleb/vgg/leaders/'
fab_repo.bs_fldr = '/data/home/shruti/voxceleb/fabnet/leaders/'

save_results(vgg_repo, fab_repo, 
             ['GG_{0:02d}'.format(f+1) for f in range(28)], 
             'real', -1, 
             'result_csv/GG_real_notime.csv', 0, 1, parallel=True)
save_results(vgg_repo, fab_repo, 
             ['GG_{0:02d}'.format(f+1) for f in range(28)], 
             'fake', -1, 
             'result_csv/GG_fake_notime.csv', 0, 1, parallel=True)

save_results(vgg_repo, fab_repo, 
             ['bo','bs','dt','ew','hc','jb'], 
             'real', 10, 
             'result_csv/ldr_real_notime.csv', 0, 1, parallel=True)
save_results(vgg_repo, fab_repo, 
             ['bo','bs','dt','ew','hc','jb'], 
             'fake', 10, 
             'result_csv/ldr_fake_notime.csv', 0, 1, parallel=True)

save_results(vgg_repo, fab_repo, 
             ['jen_l'], 
             'real', -1, 
             'result_csv/SJ_real_notime.csv', 0, 1, parallel=True)
save_results(vgg_repo, fab_repo, 
             ['steve_b'], 
             'fake', -1, 
             'result_csv/SJ_fake_notime.csv', 0, 1, parallel=True)


In [ ]:
real_df = pd.read_csv('result_csv/GG_real_notime.csv')
print(len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df))

fake_df = pd.read_csv('result_csv/GG_fake_notime.csv')
print(len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df))

In [ ]:
real_df = pd.read_csv('result_csv/ldr_real_notime.csv')
print(len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df))

fake_df = pd.read_csv('result_csv/ldr_fake_notime.csv')
print(len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df))

In [ ]:
real_df = pd.read_csv('result_csv/FF_real_notime.csv')
print(len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df))

fake_df = pd.read_csv('result_csv/FF_fake_notime.csv')
print(len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df))

In [ ]:
real_df = pd.read_csv('result_csv/SJ_real_notime.csv')
print(len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df))

fake_df = pd.read_csv('result_csv/SJ_fake_notime.csv')
print(len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df))

### Compressed Repo of Leaders + Face Forensics Original + Imposter + Google

In [ ]:
# change the location of the base folder in the repo
vgg_repo.bs_fldr = '/data/home/shruti/voxceleb/vgg/compression/'
fab_repo.bs_fldr = '/data/home/shruti/voxceleb/fabnet/compression/'

save_results(vgg_repo, fab_repo, 
             ['bo','bs','dt','ew','hc','jb'], 
             'real', 10, 
             'result_csv/ldr_real_notime_comp.csv', 0, 1, parallel=True)
save_results(vgg_repo, fab_repo, 
             ['bo','bs','dt','ew','hc','jb'], 
             'fake', 10, 
             'result_csv/ldr_fake_notime_comp.csv', 0, 1, parallel=True)



# Metric Learning

### Repo of Leaders + Face Forensics Original + Imposter + Google

In [6]:

vgg_repo_metric = get_train_repo({'istrain':True, 'bsfldr':'/data/home/shruti/voxceleb/vgg/leaders/', 
                                'frames':100, 'steps':5, 'N_comp':512})
fab_repo_metric = get_train_repo({'istrain':True, 'bsfldr':'/data/home/shruti/voxceleb/fabnet_metric/', 
                                'frames':1, 'steps':1, 'N_comp':-1})


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/opt/voxceleb/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-937ca675b52f>", line 2, in <module>
    'frames':100, 'steps':5, 'N_comp':512})
  File "<ipython-input-5-3035594aab83>", line 8, in get_train_repo
    out_repo.add_ids({f: train_files[f] for f in train_files.keys() if 'FF_' in f}, 0, 0.5)
  File "/data/home/shruti/voxceleb/motion_signature/baseline/repo.py", line 115, in add_ids
    self.emb = np.concatenate((self.emb, all_id_feat), axis=0)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/opt/voxceleb/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of 

KeyboardInterrupt: 

In [ ]:
vgg_repo_metric.bs_fldr = '/data/home/shruti/voxceleb/vgg/leaders/'
fab_repo_metric.bs_fldr = '/data/home/shruti/voxceleb/fabnet_metric/'


save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'DFDC_' in k], 
             'real', -1, 
             'result_csv/DFDC_real_100.csv', 0, 1, parallel=True)
save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'DFDC_' in k], 
             'fake', -1, 
             'result_csv/DFDC_fake_100.csv', 0, 1, parallel=True)

In [ ]:
real_df = pd.read_csv('result_csv/DFDC_real_100.csv')
print(len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df))

fake_df = pd.read_csv('result_csv/DFDC_fake_100.csv')
print(len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df))


In [ ]:

save_results(vgg_repo_metric, fab_repo_metric, 
             ['bo','bs','dt','ew','hc','jb'], 
             'real', 10, 
             'result_csv/ldr_real_100.csv', 0, 1, parallel=True)
save_results(vgg_repo_metric, fab_repo_metric, 
             ['bo','bs','dt','ew','hc','jb'], 
             'fake', 10, 
             'result_csv/ldr_fake_100.csv', 0, 1, parallel=True)

save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'FF_' in k], 
             'real', -1, 
             'result_csv/FF_real_100.csv', 0.5, 1, parallel=True)
save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'FF_' in k], 
             'fake', -1, 
             'result_csv/FF_fake_100.csv', 0.5, 1, parallel=True)

save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'GG_' in k], 
             'real', -1, 
             'result_csv/GG_real_100.csv', 0, 1, parallel=True)
save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'GG_' in k], 
             'fake', -1, 
             'result_csv/GG_fake_100.csv', 0, 1, parallel=True)

save_results(vgg_repo_metric, fab_repo_metric, 
             ['jen_l'], 
             'real', -1, 
             'result_csv/SJ_real_100.csv', 0, 1, parallel=True)
save_results(vgg_repo_metric, fab_repo_metric, 
             ['steve_b'], 
             'fake', -1, 
             'result_csv/SJ_fake_100.csv', 0, 1, parallel=True)

In [ ]:

real_df = pd.read_csv('result_csv/ldr_real_100.csv')
print(len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df))

fake_df = pd.read_csv('result_csv/ldr_fake_100.csv')
print(len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df))


real_df = pd.read_csv('result_csv/GG_real_100.csv')
print(len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df))

fake_df = pd.read_csv('result_csv/GG_fake_100.csv')
print(len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df))


In [ ]:

real_df = pd.read_csv('result_csv/FF_real_100.csv')
print(len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df))

fake_df = pd.read_csv('result_csv/FF_fake_100.csv')
print(len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df))


In [ ]:
real_df = pd.read_csv('result_csv/SJ_real_100.csv')
print(len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df))

fake_df = pd.read_csv('result_csv/SJ_fake_100.csv')
print(len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df))

### Compressed Repo of Leaders + Face Forensics Original + Imposter + Google

In [ ]:
vgg_repo_metric.bs_fldr = '/data/home/shruti/voxceleb/vgg/compression/'
fab_repo_metric.bs_fldr = '/data/home/shruti/voxceleb/fabnet_metric_compression/'

save_results(vgg_repo_metric, fab_repo_metric, 
             ['bo','bs','dt','ew','hc','jb'], 
             'real', 10, 
             'result_csv/ldr_real_100_comp.csv', 0, 1, parallel=True)
save_results(vgg_repo_metric, fab_repo_metric, 
             ['bo','bs','dt','ew','hc','jb'], 
             'fake', 10, 
             'result_csv/ldr_fake_100_comp.csv', 0, 1, parallel=True)


In [ ]:
save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['real'].keys() if 'FF_' in k], 
             'real', -1, 
             'result_csv/FF_real_100_comp.csv', 0.5, 1, parallel=True)
save_results(vgg_repo_metric, fab_repo_metric, 
             [k for k in test_files['fake'].keys() if 'FF_' in k], 
             'fake', -1, 
             'result_csv/FF_fake_100_comp.csv', 0.5, 1, parallel=True)


save_results(vgg_repo_metric, fab_repo_metric, 
             ['jen_l'], 
             'real', -1, 
             'result_csv/SJ_real_100_comp.csv', 0, 1, parallel=True)
save_results(vgg_repo_metric, fab_repo_metric, 
             ['steve_b'], 
             'fake', -1, 
             'result_csv/SJ_fake_100_comp.csv', 0, 1, parallel=True)




# file ablation for jennifer example


The accuracy of real video of length 3 minutes of Jennifer as a function of number of frames in repo for the Jennifer similar video

Also, test with close video being added to rest of the videos, or only the close video is used in the repo.

In [ ]:
# repo with leaders + ff original + imposters + Google
def build_first_repo(params):
    
    out_repo = Repo(params['bsfldr'], params['frames'], params['steps'], np.mean, 5000)
    out_repo.add_ids({f: train_files[f] for f in ['bo','br','bs','cb','dt','ew','hc','jb','kh', 'pb',
                                                  'bo_imposter','bs_imposter','ew_imposter','dt_imposter',
                                                   'hc_imposter', 'jb_imposter']}, 0, 1)
    out_repo.add_ids({'FF_{0:03d}'.format(f): train_files['FF_{0:03d}'.format(f)] for f in range(1000)}, 0, 0.5)
    out_repo.add_ids({'GG_{0:02d}'.format(f+1): train_files['GG_{0:02d}'.format(f+1)] for f in range(28)}, 0, 1)
    out_repo.add_ids({'steve_b': train_files['steve_b']}, 0, 1)
    
    return out_repo

def jennifer_ablation(out_repo, perc, both=False):
    out_repo.remove_ids(['jen_l'])
    
    if perc == 0 or both:
        out_repo.add_ids({'jen_l': [f for f in train_files['jen_l'] if 'pW7TbJJMVak' not in f]}, 0, 1)
    if perc > 0:
        out_repo.add_ids({'jen_l': ['jennifer_l/pW7TbJJMVak_0.npy']}, 0, perc)
        
    out_repo.build_repo_noKDD()

def run_one_case(vgg_repo, fab_repo, perc, both=False):
    
    jennifer_ablation(vgg_repo, perc, both)
    jennifer_ablation(fab_repo, perc, both)
    
    real_df = save_results(vgg_repo, fab_repo, ['jen_l'], 'real', -1, None, 0, 1, parallel=True)
    fake_df = save_results(vgg_repo, fab_repo, ['steve_b'], 'fake', -1, None, 0, 1, parallel=True)
    
    real_acc = len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df)
    fake_acc = len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df)
    
    return real_acc, fake_acc


def jennifer_file_ablation(out_repo, file):
    out_repo.remove_ids(['jen_l'])
    out_repo.add_ids({'jen_l': file}, 0, 1)
    out_repo.build_repo_noKDD()
    
    
def run_one_file_ablation_case(vgg_repo, fab_repo, file):
    
    jennifer_file_ablation(vgg_repo, file)
    jennifer_file_ablation(fab_repo, file)
    
    real_df = save_results(vgg_repo, fab_repo, ['jen_l'], 'real', -1, None, 0, 1, parallel=True)
    fake_df = save_results(vgg_repo, fab_repo, ['steve_b'], 'fake', -1, None, 0, 1, parallel=True)
    
    real_acc = len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df)
    fake_acc = len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df)
    
    return real_acc, fake_acc
    
    

In [ ]:
vgg_repo = build_first_repo({'istrain':True, 'bsfldr':'/data/home/shruti/voxceleb/vgg/leaders/', 
                                'frames':100, 'steps':5, 'N_comp':512})
fab_repo = build_first_repo({'istrain':True, 'bsfldr':'/data/home/shruti/voxceleb/fabnet_metric/', 
                                'frames':1, 'steps':1, 'N_comp':-1})

In [ ]:
jennifer_ablation(vgg_repo, 0, both=False)
jennifer_ablation(fab_repo, 0, both=False)

real_df = save_results(vgg_repo, fab_repo, ['jen_l'], 'real', -1, 'result_csv/SJ_real_100_onlyother.csv', 0, 1, parallel=True)
fake_df = save_results(vgg_repo, fab_repo, ['steve_b'], 'fake', -1, 'result_csv/SJ_fake_100_onlyother.csv', 0, 1, parallel=True)


In [ ]:
#Case1 only other videos
r1, f1 = run_one_case(vgg_repo, fab_repo, 0, both=False)

#Case2 other videos with 3 minutes new video
r2, f2 = run_one_case(vgg_repo, fab_repo, 1, both=False)

#Case3 only overlapping video with diff perc
p_rng = np.linspace(0.03, 1, 10)
r_f3 = np.zeros((len(p_rng), 3))
for i in range(len(p_rng)):
    r_f3[i, 0] = 3*60*p_rng[i] # these many seconds
    r_f3[i, 1:] = run_one_case(vgg_repo, fab_repo, p_rng[i], both=False)

#Case4 both videos with diff perc
r_f4 = np.zeros((len(p_rng), 3))
for i in range(len(p_rng)):
    r_f4[i, 0] = 3*60*p_rng[i] # these many seconds
    r_f4[i, 1:] = run_one_case(vgg_repo, fab_repo, p_rng[i], both=True)



In [ ]:

files_jen = np.unique([f.split('/')[-1].split('_')[0] for f in train_files['jen_l']])
print(files_jen)

acc = np.zeros((len(files_jen), 2))
for i in range(len(files_jen)):
    
    cur_files = [f for f in train_files['jen_l'] if files_jen[i] in f]
    acc[i, :] = run_one_file_ablation_case(vgg_repo, fab_repo, cur_files)
    

In [ ]:
result_df = pd.DataFrame(data = acc, columns=['real_acc', 'fake_acc'])
result_df['filename'] = files_jen
result_df['avg_acc'] = (np.array(result_df['real_acc']) + np.array(1-result_df['fake_acc']))/2

display(result_df.sort_values('avg_acc'))

In [ ]:
# plot the results
print(f'Only other videos: \t \t Real: {r1:0.3f} Fake: {f1:0.3f}')

print(f'3min Similar: \t \t Real: {r2:0.3f} Fake: {f2:0.3f}')

plt.plot([0] + list(r_f3[:, 0]), [r1*100] + list(r_f3[:, 1]*100), label='real')
plt.plot([0] + list(r_f3[:, 0]), [f1*100] + list(r_f3[:, 2]*100), label='fake')
plt.xlabel('Time (Sec)')
plt.ylabel('Acc')
plt.grid(b=True)
plt.legend()
plt.title('T of Similar Vid')
plt.show()


plt.plot([0] + list(r_f4[:, 0]), [r1*100] + list(r_f4[:, 1]*100), label='real')
plt.plot([0] + list(r_f4[:, 0]), [f1*100] + list(r_f4[:, 2]*100), label='fake')
plt.xlabel('Time (Sec)')
plt.ylabel('Acc')
plt.grid(b=True)
plt.legend()
plt.title('Other + T of Similar Vid')
plt.show()


In [ ]:

x = [25, 50, 75, 100]
y = [86.5, 91.7, 94.7, 95.9]

x_nm = [1, 25]
y_nm = [43.1, 50.44]

plt.figure(figsize=(8,8))

plt.plot(x, y, label='with metric')
plt.plot(x_nm, y_nm, label='no metric')

plt.xlabel('Frames')
plt.ylabel('Recall@1')
plt.grid(b=True)
plt.title('voxceleb recall')
plt.legend()
plt.show()


# performance of diff length metric learning

In [ ]:

def run_one_time(N):
    
    vgg_repo_metric = get_train_repo({'istrain':True, 'bsfldr':'/data/home/shruti/voxceleb/vgg/leaders/', 
                                    'frames':100, 'steps':5, 'N_comp':512})
    fab_repo_metric = get_train_repo({'istrain':True, 'bsfldr':f'/data/home/shruti/voxceleb/fabnet_metric{N}/', 
                                    'frames':1, 'steps':1, 'N_comp':-1})

    save_results(vgg_repo_metric, fab_repo_metric, 
                 ['bo','bs','dt','ew','hc','jb'], 
                 'real', 10, 
                 f'result_csv/ldr_real_{N}.csv', 0, 1, parallel=True)
    save_results(vgg_repo_metric, fab_repo_metric, 
                 ['bo','bs','dt','ew','hc','jb'], 
                 'fake', 10, 
                 f'result_csv/ldr_fake_{N}.csv', 0, 1, parallel=True)

    save_results(vgg_repo_metric, fab_repo_metric, 
                 ['FF_{0:03d}'.format(f) for f in range(1000)], 
                 'real', -1, 
                 f'result_csv/FF_real_{N}.csv', 0.5, 1, parallel=True)
    save_results(vgg_repo_metric, fab_repo_metric, 
                 ['FF_{0:03d}'.format(f) for f in range(1000)], 
                 'fake', -1, 
                 f'result_csv/FF_fake_{N}.csv', 0.5, 1, parallel=True)

    save_results(vgg_repo_metric, fab_repo_metric, 
                 ['GG_{0:02d}'.format(f+1) for f in range(28)], 
                 'real', -1, 
                 f'result_csv/GG_real_{N}.csv', 0, 1, parallel=True)
    save_results(vgg_repo_metric, fab_repo_metric, 
                 ['GG_{0:02d}'.format(f+1) for f in range(28)], 
                 'fake', -1, 
                 f'result_csv/GG_fake_{N}.csv', 0, 1, parallel=True)

    save_results(vgg_repo_metric, fab_repo_metric, 
                 ['jen_l'], 
                 'real', -1, 
                 f'result_csv/SJ_real_{N}.csv', 0, 1, parallel=True)
    save_results(vgg_repo_metric, fab_repo_metric, 
                 ['steve_b'], 
                 'fake', -1, 
                 f'result_csv/SJ_fake_{N}.csv', 0, 1, parallel=True)

    real_df = pd.read_csv(f'result_csv/ldr_real_{N}.csv')
    print(len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df))

    fake_df = pd.read_csv(f'result_csv/ldr_fake_{N}.csv')
    print(len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df))

    real_df = pd.read_csv(f'result_csv/GG_real_{N}.csv')
    print(len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df))

    fake_df = pd.read_csv(f'result_csv/GG_fake_{N}.csv')
    print(len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df))

    real_df = pd.read_csv(f'result_csv/FF_real_{N}.csv')
    print(len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df))

    fake_df = pd.read_csv(f'result_csv/FF_fake_{N}.csv')
    print(len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df))

    real_df = pd.read_csv(f'result_csv/SJ_real_{N}.csv')
    print(len(np.argwhere(np.array(real_df['predLabel_vgg'])==np.array(real_df['predLabel_fab'])).ravel())/len(real_df))

    fake_df = pd.read_csv(f'result_csv/SJ_fake_{N}.csv')
    print(len(np.argwhere(np.array(fake_df['predLabel_vgg'])==np.array(fake_df['predLabel_fab'])).ravel())/len(fake_df))


In [ ]:
run_one_time(25)
run_one_time(50)
run_one_time(75)

# test face forensics on these videos
